<div style="
  font-family: 'Courier New', monospace;
  font-size: 3.5rem;
  font-weight: bold;
  color: #00ff00;
  text-shadow: 
    0 0 5px #00ff00, 
    0 0 10px #00ff00, 
    0 0 20px #00ff00,
    0 0 40px #00ff00;
  background: #000;
  padding: 20px;
  border-radius: 5px;
  border: 2px solid #00ff00;
  display: inline-block;
  box-shadow: 
    0 0 10px rgba(0, 255, 0, 0.5),
    0 0 20px rgba(0, 255, 0, 0.3);
  position: relative;
  overflow: hidden;
">

  <!-- Glitch Effect -->
  <div style="
    position: absolute;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    background: linear-gradient(
      rgba(0, 255, 0, 0.1),
      rgba(0, 255, 0, 0.05)
    );
    animation: glitch 1s infinite;
  "></div>

  <!-- Text -->
  <span style="position: relative; z-index: 1;">ColaBit</span>

  <!-- Matrix Falling Code Effect -->
  <div style="
    position: absolute;
    top: -100%;
    left: 0;
    width: 100%;
    height: 100%;
    background: linear-gradient(
      transparent,
      rgba(0, 255, 0, 0.1),
      transparent
    );
    animation: matrix 3s linear infinite;
  "></div>
</div>

<style>
  @keyframes glitch {
    0% { transform: translate(0); }
    20% { transform: translate(-2px, 2px); }
    40% { transform: translate(2px, -2px); }
    60% { transform: translate(-2px, 2px); }
    80% { transform: translate(2px, -2px); }
    100% { transform: translate(0); }
  }

  @keyframes matrix {
    0% { top: -100%; }
    100% { top: 100%; }
  }
</style>

<div style="
  font-family: 'Arial', sans-serif;
  font-size: 1.2rem;
  color: #ffffff;
  background: #1e1e1e;
  padding: 15px 25px;
  border-radius: 8px;
  border-left: 5px solid #4caf50;
  box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
  display: inline-block;
">
  <span style="color: #4caf50;">$</span> install dependencies
</div>

In [11]:
%%capture

# Install and update essential tools for managing Python packages
get_ipython().system('python -m pip install --upgrade pip setuptools wheel')

# Install libraries for downloading and handling torrent files
get_ipython().system('python -m pip install lbry-libtorrent wget torf')

# Install system-level dependency for torrent functionality
get_ipython().system('apt install python3-libtorrent -y')

<div style="
  font-family: 'Arial', sans-serif;
  font-size: 1.2rem;
  color: #ffffff;
  background: #1e1e1e;
  padding: 15px 25px;
  border-radius: 8px;
  border-left: 5px solid #4caf50;
  box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
  display: inline-block;
">
  <span style="color: #4caf50;">$</span> download the torrent
</div>

In [10]:
import zipfile
import os
import libtorrent as lt
import time
import datetime
import shutil

# --- Torrent Download Settings ---
DOWNLOAD_PATH = '/content/Torrent/'
STORAGE_MODE = lt.storage_mode_t(2)

params = {
    'save_path': DOWNLOAD_PATH,
    'storage_mode': STORAGE_MODE,
}

# --- Start Libtorrent Session ---
ses = lt.session()
ses.listen_on(6881, 6891)

# --- Get Torrent/Magnet Link from User ---
link = input("Enter Torrent file link or Magnet link: ")

# --- Handle Torrent File Conversion ---
if link.endswith('.torrent'):
    import wget
    from torf import Torrent

    TORRENT_FILE = 'torrent.torrent'

    if os.path.exists(TORRENT_FILE):
        os.remove(TORRENT_FILE)

    wget.download(link, TORRENT_FILE)
    torrent_info = Torrent.read(TORRENT_FILE)
    link = str(torrent_info.magnet(name=True, size=False, trackers=False, tracker=False))

# --- Start Download ---
print("Using link:", link)
handle = lt.add_magnet_uri(ses, link, params)
handle.set_sequential_download(0)  # Download in parallel
ses.start_dht()

begin = time.time()
print(datetime.datetime.now())
print("Downloading Metadata...")

while not handle.has_metadata():
    time.sleep(1)

print("Got Metadata, Starting Torrent Download...")
print("Starting:", handle.name())

# --- Monitor and Display Progress ---
while handle.status().state != lt.torrent_status.seeding:
    status = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata',
                 'downloading', 'finished', 'seeding', 'allocating']

    print(
        '%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' %
        (status.progress * 100, status.download_rate / 1000,
         status.upload_rate / 1000, status.num_peers, state_str[status.state])
    )
    time.sleep(5)

# --- Download Complete ---
end = time.time()
print(handle.name(), "COMPLETE")
print("Elapsed Time: ", int((end - begin) // 60), "min :", int((end - begin) % 60), "sec")
print(datetime.datetime.now())

<div style="
  font-family: 'Arial', sans-serif;
  font-size: 1.2rem;
  color: #ffffff;
  background: #1e1e1e;
  padding: 15px 25px;
  border-radius: 8px;
  border-left: 5px solid #4caf50;
  box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
  display: inline-block;
">
  <span style="color: #4caf50;">$</span> zip the downloaded files
</div>

In [12]:
import zipfile
import os

def zip_downloaded_torrent(torrent_directory='content/Torrent/', zip_filename='Downloaded-Torrent.zip'):
    """
    Zips the contents of the downloaded torrent directory into a single ZIP file.

    Args:
        torrent_directory: The path to the directory containing the downloaded torrent files.
        zip_filename: The name of the ZIP file to be created.
    """
    print('Zipping started...')

    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        for root, _, files in os.walk(torrent_directory):
            for file in files:
                # Construct the full file path
                file_path = os.path.join(root, file)

                # Calculate the relative path for archiving
                archive_path = os.path.relpath(file_path, os.path.join(torrent_directory, '..'))

                # Add the file to the ZIP archive
                zip_file.write(file_path, archive_path)

    print('\nALL DONE!')

# Call the function to zip the downloaded torrent
zip_downloaded_torrent()